In [0]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [0]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico - Segmentação de Risco

### Geração de Massa

In [0]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'payloadHomol.intermediarias.recorrenciaPrincipalidade':[0],
                             'payloadHomol.intermediarias.flagClienteExistente':[0],
                             'solicitante.saldoMinimo30d':[0],
                             'payloadHomol.intermediarias.MATRecorrente':[0],
                             'solicitante.flagMAT':[0]
                            })

In [0]:
#Criação das massas com casos forçados
#Digito CPF Concomitante CP
massa_SegmentacaoRisco = sherlock.Gera_Massa(base_entrada)
massa_SegmentacaoRisco.gera_cenario('payloadHomol.intermediarias.recorrenciaPrincipalidade',[0,1])
massa_SegmentacaoRisco.gera_cenario('payloadHomol.intermediarias.flagClienteExistente',[0,1])
massa_SegmentacaoRisco.gera_cenario('solicitante.saldoMinimo30d',[-99999,0,99,99.99,100,100.1,101,])
massa_SegmentacaoRisco.gera_cenario('payloadHomol.intermediarias.MATRecorrente',[0,1])
massa_SegmentacaoRisco.gera_cenario('solicitante.flagMAT',[0,1])

massa_SegmentacaoRisco.get_massa_final()

massa_SegmentacaoRisco.set_bom_arqv()
massa_SegmentacaoRisco.converte_massa()

### Execução Modular e Geração de CSV

In [0]:
listaResultado = rp.execucaoModular(massa_SegmentacaoRisco.get_massa_json(),rp.SegmentacaoRisco)

##### Valida se todas condições estão sendo abordadas

In [0]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['segmentacaoRisco'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [0]:
# Renomeação das colunas para o cabeçalho do RMA
massa_SegmentacaoRisco.rename_cabecalho()

In [0]:
massa_SegmentacaoRisco.atribui_expected(listaResultado,"payloadHomol.intermediarias.segmentacaoRisco","segmentacaoRisco")
massa_SegmentacaoRisco.get_massa_dm()

### Conversão da massa para CSV

In [0]:
massa_SegmentacaoRisco.get_massa_dm().to_csv(r'/Workspace/Users/kaue.ongaro@picpaybank.com/2 - Homologação/Concessao de Cartoes - Online/massas/massaSegmentacaoRisco.csv',index=False)